# Revista de Gestão, Finanças e Contabilidade

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(url_search):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find_all('div', {'class': 'title'})
    a_links = []
    for item in results:
        try:
            remover_links = 'Editorial' in item.find('a').text
            remover_links = remover_links or 'Folha de rosto' in item.find('a').text
            remover_links = remover_links or 'Dados da edição ' in item.find('a').text
            if not remover_links:
                a_links.append(item.find('a')['href'])
        except:
            pass
    return a_links

In [4]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(url_search)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links

In [5]:
url_search_text = 'https://revistas.uneb.br/index.php/financ/search/search?query=__query__&searchJournal=3&authors=&title=&abstract=&galleyFullText=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&orderBy=score&orderDir=desc&searchPage=__pg_num__#results'

links = listar_links(url_search_text, 'contabilidade+ambiental', pg_num=1)

link = links[0]

link


'https://revistas.uneb.br/index.php/financ/article/view/740'

In [6]:
links

['https://revistas.uneb.br/index.php/financ/article/view/740',
 'https://revistas.uneb.br/index.php/financ/article/view/7376',
 'https://revistas.uneb.br/index.php/financ/article/view/3143',
 'https://revistas.uneb.br/index.php/financ/article/view/37',
 'https://revistas.uneb.br/index.php/financ/article/view/5514',
 'https://revistas.uneb.br/index.php/financ/article/view/255',
 'https://revistas.uneb.br/index.php/financ/article/view/13401']

In [7]:
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [8]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [9]:
def limpar_afiliacao(afiliacao):
    afiliacao = ' '.join(afiliacao.split())
    afiliacao = afiliacao.split('-')[0]
    afiliacao = afiliacao.split('(')[0]
    return afiliacao.strip()

In [10]:
def captura_titulo(soup):
    try:
        title = soup.find('h1', {'class': 'page_title'}).text
        title = ' '.join(title.split())
    except:
        title = ''
    return {'Título': title}

In [11]:
captura_titulo(soup)

{'Título': 'GESTÃO DA SUSTENTABILIDADE: UM ESTUDO SOBRE O NÍVEL DE SUSTENTABILIDADE SOCIOAMBIENTAL DE UMA INSTITUIÇÃO FEDERAL DE ENSINO SUPERIOR'}

In [12]:
def captura_data(soup):
    try:
        date_tag = soup.find('div', {'class': 'item published'})
        date = date_tag.find('div', {'class': 'value'}).text
        date = ' '.join(date.split())
    except:
        date = ''
    return {'Data de Publicação': date[:10]}

In [13]:
captura_data(soup)

{'Data de Publicação': '2014-12-01'}

In [14]:
def captura_resumo(soup):
    try:
        abstract_section = soup.find('div', {'class': 'item abstract'})
        abstract = abstract_section.text
        abstract = abstract.replace('Resumo:', '')
        abstract = abstract.replace('Resumo', '').strip()
        #abstract = ' '.join(abstract.split())
    except:
        abstract = ''
    return {'Resumo': abstract}

In [15]:
captura_resumo(soup)

{'Resumo': 'O objetivo deste estudo é avaliar o nível de sustentabilidade ambiental na Universidade Federal Fronteira Sul, campi de Chapecó/SC, a partir da aplicação do Modelo de Avaliação de Sustentabilidade Socioambiental (MASS) elaborado por Freitas (2013). Para o alcance do objetivo proposto, aplicou-se o questionário Educação Ambiental com 229 questões fechadas, respondido pelo departamento de sustentabilidade da UFFS. Os dados foram tabulados no software phinx e tratados de maneira quantitativa com o auxílio da planilha eletrônica Excel e do software, utilizado para a elaboração das redes. Os resultados demonstram que UFFS apresenta 53,09% do índice geral de sustentabilidade atingido (numa escala de 0 a 100%), representando que a instituição aplica a Sustentabilidade Ambiental preponderantemente para cumprir os requisitos legais.'}

In [16]:
def captura_palavras_chave(soup):
    try:
        kw_section = soup.find('div', {'class': 'item keywords'})
        kw = kw_section.text.replace('Palavras-chave:', '')
        kw = ' '.join(kw.split())
        kw = kw.replace(',', '.').replace(';', '.')
    except:
        kw = ''
    if kw == '':
        try:
            kw_section = soup.find('section', {'class': 'item abstract'})
            p_list = kw_section.find_all('p')
            for p in p_list:
                if 'Palavras-chave' in p.text:
                    kw_full = p.text
                    kw = kw_full.replace('Palavras-chave', '').replace(':', '')
                    kw = ' '.join(kw.split())
                    kw = kw.replace(',', '.').replace(';', '.')
        except:
            pass
    return {'Palavras-chave': kw}

In [17]:
captura_palavras_chave(soup)

{'Palavras-chave': 'Sustentabilidade Ambiental. Instituição de Ensino Superior. Socioambiental. Gestão Sustentável. Nível de Sustentabilidade.'}

In [18]:
def captura_autores(soup):
    autores = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        autores = authors_ul.find_all('span', {'class': 'name'})
        autores = [' '.join(a.text.split()) for a in autores]
    except:
        pass
    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [19]:
captura_autores(soup)

{'Qtd. de Autores': 3,
 'Autor 1': 'Ines Liani Menzel Warken',
 'Autor 2': 'Veridiana Jéssica Henn',
 'Autor 3': 'Fabricia Silva da Rosa',
 'Autor 4': '',
 'Autor 5': '',
 'Autor 6': ''}

In [20]:
def captura_afiliacao(soup):
    afiliacao = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        afiliacao = authors_ul.find_all('span', {'class': 'affiliation'})
        afiliacao = [' '.join(a.text.split()) for a in afiliacao]
        afiliacao = [limpar_afiliacao(a) for a in afiliacao]
    except:
        pass
    afiliacao = completar_lista(afiliacao)
    
    key_affiliation = [f"Afiliação {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, afiliacao))

In [21]:
captura_afiliacao(soup)

{'Afiliação 1': 'Universidade Regional de Blumenau',
 'Afiliação 2': 'HORUS FACULDADES',
 'Afiliação 3': 'Universidade Regional de Blumenau',
 'Afiliação 4': '',
 'Afiliação 5': '',
 'Afiliação 6': ''}

In [22]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'Revista de Gestão, Finanças e Contabilidade'}
    url_search_text = 'https://revistas.uneb.br/index.php/financ/search/search?query=__query__&searchJournal=3&authors=&title=&abstract=&galleyFullText=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&orderBy=score&orderDir=desc&searchPage=__pg_num__#results'
    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'Url Artigo': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        dict_artigo = {
            **captura_data(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%Y-%m-%d')
    return df

## Pesquisa por artigos

In [23]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head(3)

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2014-12-01,GESTÃO DA SUSTENTABILIDADE: UM ESTUDO SOBRE O ...,O objetivo deste estudo é avaliar o nível de s...,Sustentabilidade Ambiental. Instituição de Ens...,3,Ines Liani Menzel Warken,Veridiana Jéssica Henn,Fabricia Silva da Rosa,,,,Universidade Regional de Blumenau,HORUS FACULDADES,Universidade Regional de Blumenau,,,,https://revistas.uneb.br/index.php/financ/arti...,"Revista de Gestão, Finanças e Contabilidade"
1,2019-12-21,DEMONSTRAÇÃO DO VALOR ADICIONADO (DVA): UMA AN...,Esse trabalho teve como objetivo analisar as D...,Demonstração do Valor Adicionado. Índice de Su...,2,Thaís Santos de Sousa,Juliano Almeida de Faria,,,,,Universidade Estadual de Feira de Santana,Universidade Federal de Sergipe,,,,,https://revistas.uneb.br/index.php/financ/arti...,"Revista de Gestão, Finanças e Contabilidade"
2,2017-03-14,PRODUÇÃO ACADÊMICA BRASILEIRA NO CONTEXTO DOS ...,Com o intuito de contribuir para a evolução e ...,Sustentabilidade. Relatórios de Sustentabilida...,5,Caroline Moya de Moraes,Gabrielli Aparecida Zevericoski,Jaqueline dos Santos Ferrarezi,Karina Rocha Henriques Gehlen,Luciano Gomes dos Reis,,Universidade Estadual de Londrina,Universidade Estadual de Londrina,Universidade Estadual de Londrina,Universidade Estadual de Londrina,Universidade Estadual de Londrina,,https://revistas.uneb.br/index.php/financ/arti...,"Revista de Gestão, Finanças e Contabilidade"


In [24]:
df.to_csv('../data/RGFC.csv', index=False)

In [25]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2014-12-01,GESTÃO DA SUSTENTABILIDADE: UM ESTUDO SOBRE O ...,O objetivo deste estudo é avaliar o nível de s...,Sustentabilidade Ambiental. Instituição de Ens...,3,Ines Liani Menzel Warken,Veridiana Jéssica Henn,Fabricia Silva da Rosa,,,,Universidade Regional de Blumenau,HORUS FACULDADES,Universidade Regional de Blumenau,,,,https://revistas.uneb.br/index.php/financ/arti...,"Revista de Gestão, Finanças e Contabilidade"
1,2019-12-21,DEMONSTRAÇÃO DO VALOR ADICIONADO (DVA): UMA AN...,Esse trabalho teve como objetivo analisar as D...,Demonstração do Valor Adicionado. Índice de Su...,2,Thaís Santos de Sousa,Juliano Almeida de Faria,,,,,Universidade Estadual de Feira de Santana,Universidade Federal de Sergipe,,,,,https://revistas.uneb.br/index.php/financ/arti...,"Revista de Gestão, Finanças e Contabilidade"
2,2017-03-14,PRODUÇÃO ACADÊMICA BRASILEIRA NO CONTEXTO DOS ...,Com o intuito de contribuir para a evolução e ...,Sustentabilidade. Relatórios de Sustentabilida...,5,Caroline Moya de Moraes,Gabrielli Aparecida Zevericoski,Jaqueline dos Santos Ferrarezi,Karina Rocha Henriques Gehlen,Luciano Gomes dos Reis,,Universidade Estadual de Londrina,Universidade Estadual de Londrina,Universidade Estadual de Londrina,Universidade Estadual de Londrina,Universidade Estadual de Londrina,,https://revistas.uneb.br/index.php/financ/arti...,"Revista de Gestão, Finanças e Contabilidade"
3,2012-06-12,SUSTENTABILIDADE AMBIENTAL: UM ESTUDO NA AUTOR...,Mesmo com a grande relevância dos portos para ...,Gestão Ambiental. Sustentabilidade. Portos.,5,ARTURO GINER FILLOL,FABRICIA SILVA ROSA,ROGERIO JOAO LUNKES,VICENTE MATEO RIPOLL FELIU,CRISTINA CRESPO SOLER,,Autoridade Portuária de Valéncia – España,UFSC,UFSC,UNIVERSITAT DE VALENCIA,Universitat de Valencia,,https://revistas.uneb.br/index.php/financ/arti...,"Revista de Gestão, Finanças e Contabilidade"
4,2019-12-21,RELAÇÃO ENTRE O GERENCIAMENTO TRIBUTÁRIO E A E...,Esta pesquisa objetivou avaliar a relação entr...,Passivo Contingente Tributário. Gerenciamento ...,2,Rúbia Albers Magalhães,Luiz Felipe Ferreira,,,,,Programa de Pós,Universidade Federal de Santa Catarina,,,,,https://revistas.uneb.br/index.php/financ/arti...,"Revista de Gestão, Finanças e Contabilidade"
5,2013-08-26,CARACTERÍSTICAS DO CONSELHO DE ADMINISTRAÇÃO E...,O conceito de Responsabilidade Social Corporat...,Responsabilidade Social Corporativa. Divulgaçã...,2,Maria de Fátima Matos,Cristina Gonçalves Góis,,,,,Mestre em Contabilidade e Fiscalidade Empresarial,ISCAC,,,,,https://revistas.uneb.br/index.php/financ/arti...,"Revista de Gestão, Finanças e Contabilidade"
6,2021-12-29,EDITORIAL,Prezados(as) leitores(as):\nEste segundo volum...,,3,Ricardo José Rocha Amorim,Francisco Marton Gleuson Pinheiro,Maria de Fátima Araújo Frazão,,,,Universidade do Estado da Bahia,Universidade do Estado da Bahia,Universidade do Estado da Bahia,,,,https://revistas.uneb.br/index.php/financ/arti...,"Revista de Gestão, Finanças e Contabilidade"
7,2017-03-14,PRODUÇÃO ACADÊMICA BRASILEIRA NO CONTEXTO DOS ...,Com o intuito de contribuir para a evolução e ...,Sustentabilidade. Relatórios de Sustentabilida...,5,Caroline Moya de Moraes,Gabrielli Aparecida Zevericoski,Jaqueline dos Santos Ferrarezi,Karina Rocha Henriques Gehlen,Luciano Gomes dos Reis,,Universidade Estadual de Londrina,Universidade Estadual de Londrina,Universidade Estadual de Londrina,Universidade Estadual de Londrina,Universidade Estadual de Londrina,,https://revistas.uneb.br/index.php/financ/arti...,"Revista de Gestão, Finanças e Contabilidade"
8,2016-05-17,GOVERNANÇA CORPORATIVA NA ESTRUTURA CONCEITUAL...,O presente artigo verificou como as empresas b...,Governança Corporativa. Relato integrado. Estr...,5,Ana Cristina Silva Abreu,Elise Soerger Zaro,Guilherme Luiz,Ernesto Fernando Rodrigues Vicente,Hans Mich